# Queue

In [9]:
import queue

# queue 생성
q1 = queue.Queue()# 큐의 크기가 무한데
q2 = queue.Queue(100)# 큐의 크기가 100

#queue 자료를 넣으려면

q1.put('spam')
print(q1.qsize())
# 큐에 데이터를 넣고, 차 있으면 대기한다
q1.put_nowait('ham')
print(q1.qsize())

#Queue 에서 자료를 꺼내려면
print(q1.get())
print(q1.qsize())
print(q1.get_nowait())

#Queue 상태를 알려주는 메소드
q1.qsize()
q1.empty()
q1.full()


1
2
spam
1
ham


False

# Thread

In [20]:
from threading import Thread
import time

def my_thread(val):
    for i in range(3):
        print("I'm Thread")
        time.sleep(1)
##인스턴스 만들기
## 첫번째 argu 는 스레드 함수 이름, 두번째 argu는 매개변수를 투풀형태로 전달한 것
t1=Thread(target = my_thread,args=(1,))

##스레드 시작.
t1.start()
# t1.join()#t1 thread가 종료될 때까지 기다림

## main

for i in range(3):
    print("i'm main Thread")
    time.sleep(1)
print('----=end=----')

I'm Threadi'm main Thread

i'm main Thread
I'm Thread
i'm main Thread
I'm Thread
----=end=----


In [23]:
!pip install feedparser

  Stored in directory: C:\Users\82102\AppData\Local\pip\Cache\wheels\8c\69\b7\f52763c41c5471df57703a0ef718a32a5e81ee35dcf6d4f97f
Successfully built feedparser


In [33]:
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')
import threading
import queue
import feedparser
import uuid
threads = []
queues = [queue.Queue(), queue.Queue()]
def extractWords():
    url = 'https://timesofindia.indiatimes.com/rssfeeds/1081479906.cms'
    feed = feedparser.parse(url)
    for entry in feed['entries'][:5]:
        text = entry['title']
        if 'ex' in text:
            continue
        words = nltk.word_tokenize(text)
        data = {'uuid': uuid.uuid4(), 'input': words}
        queues[0].put(data, True)
        print(">> {} : {}".format(data['uuid'], text))
def extractPOS():
    while True:
        if queues[0].empty():
            break
        else:
            data = queues[0].get()
            words = data['input']
            postags = nltk.pos_tag(words)
            queues[0].task_done()
            queues[1].put({'uuid': data['uuid'], 'input': postags}, True)
def extractNE():
    while True:
        if queues[1].empty():
            break
        else:
            data = queues[1].get()
            postags = data['input']
            queues[1].task_done()
            chunks = nltk.ne_chunk(postags, binary=False)
            print(" << {} : ".format(data['uuid']), end = '')
            for path in chunks:
                try:
                    label = path.label()
                    print(path, end=', ')
                except:
                    pass
            print()
def runProgram():
    e = threading.Thread(target=extractWords())
    e.start()
    threads.append(e)
    p = threading.Thread(target=extractPOS())
    p.start()
    threads.append(p)
    n = threading.Thread(target=extractNE())
    n.start()
    threads.append(n)
    queues[0].join()
    queues[1].join()
#print(queues[0].empty())
#print(queues[1].empty())
for t in threads:
    t.join()
if __name__ == '__main__':
    runProgram()

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\82102\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     C:\Users\82102\AppData\Roaming\nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\82102\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


>> 4d5b9553-0bdf-45e7-b750-214370165ead : Watch: Journey of Hrithik's Super 30 students
>> 6e18f14e-5590-480f-a08c-7e6f665fc479 : Batla House song Rula Diya to be unveiled tom
>> ac0de41d-d00c-4f04-ad1f-4a6d92a7bf50 : Rahul on being charged Rs 442 for bananas
>> af4565e0-8a13-4c52-8191-4196dc0b397c : Taimur and Inaaya's fun day out in London
>> d82942cd-cad5-41c0-8330-2cc9d11212b7 : Female action heroes take lead in Hollywood
 << 4d5b9553-0bdf-45e7-b750-214370165ead : (GPE Watch/NN), (GPE Hrithik/NNP), 
 << 6e18f14e-5590-480f-a08c-7e6f665fc479 : (PERSON Batla/NNP), (ORGANIZATION House/NNP), (PERSON Rula/NNP Diya/NNP), 
 << ac0de41d-d00c-4f04-ad1f-4a6d92a7bf50 : (GPE Rahul/NN), 
 << af4565e0-8a13-4c52-8191-4196dc0b397c : (GPE Taimur/NNP), (PERSON Inaaya/NNP), (GPE London/NNP), 
 << d82942cd-cad5-41c0-8330-2cc9d11212b7 : (GPE Female/JJ), (GPE Hollywood/NNP), 


# 유사도

In [34]:
word_embedding_dic = {
'사과' : [1.0, 0.5],
'바나나' : [0.9, 1.2],
'원숭이' : [0.5, 1.5]
}

In [35]:
import numpy as np
def euclidean_dist(x,y):
    x = np.array(x)
    y = np.array(y)
    return np.sqrt(np.sum(x-y)**2)

euclidean_dist(word_embedding_dic['사과'],word_embedding_dic['바나나'])

0.6

In [37]:
s1 = '대부분 원숭이는 바나나를 좋아합니다.'
s2 = '코주부 원숭이는 바나나를 싫어합니다.'

token_s1 = s1.split()
token_s2 = s2.split()

union = set(token_s1).union(set(token_s2))
print(union)


intersection = set(token_s1).intersection(set(token_s2))
print(intersection)

print(len(intersection)/len(union))

{'원숭이는', '좋아합니다.', '대부분', '싫어합니다.', '바나나를', '코주부'}
{'원숭이는', '바나나를'}
0.3333333333333333


In [38]:
def cosine_similarity(x,y):
    nominator = np.dot(x,y)
    denominator = np.linalg.norm(x)*np.linalg.norm(y)
    return nominator/denominator

a= np.array([1,2])
b= np.array([3,4])
np.dot(a,b)

11

In [40]:
sentence_ls = [
'오늘 동물원에서 코끼리를 봤어'
,
'오늘 동물원에서 원숭이에게 사과를 줬어'
]

In [41]:
sentence_ls = [sentence.split() for sentence in sentence_ls]

sentence_ls

[['오늘', '동물원에서', '코끼리를', '봤어'], ['오늘', '동물원에서', '원숭이에게', '사과를', '줬어']]

In [43]:
from collections import defaultdict

token_dict = defaultdict(lambda : len(token_dict))

for sentence in sentence_ls:
    for token in sentence:
        token_dict[token]

In [45]:
index_token_ls = sorted((value,key) for key,value in token_dict.items())
index_token_ls

[(0, '오늘'),
 (1, '동물원에서'),
 (2, '코끼리를'),
 (3, '봤어'),
 (4, '원숭이에게'),
 (5, '사과를'),
 (6, '줬어')]

In [46]:
token_in_order = [tup[1] for tup in index_token_ls]
token_in_order

['오늘', '동물원에서', '코끼리를', '봤어', '원숭이에게', '사과를', '줬어']

In [47]:
import pandas as pd
import numpy as np
n_words  = len(token_dict)
n_sentence = len(sentence_ls)

BOW = pd.DataFrame(np.zeros((n_sentence,n_words)),columns = token_in_order,index = ['문장_1','문장_2'],dtype = int)

In [48]:
for i,sentence in enumerate(sentence_ls):
    for token in sentence:
        token_location = token_dict[token]
        BOW.iloc[i,token_location] +=1

In [49]:
BOW

,오늘,동물원에서,코끼리를,봤어,원숭이에게,사과를,줬어
문장_1,1,1,1,1,0,0,0
문장_2,1,1,0,0,1,1,1
